In [16]:
import sys

sys.path.append('..')
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
names = ['tiny', 'tiny_ft', 'tiny_ss', 'tiny_ss_ft', 'tiny_ss_ft_freeze', 'tiny_ft_freeze']
metrics = ['loss', 'accuracy', 'precision', 'recall', 'f1', 'wer']

# get results for each model
results = {}
for name in names:
  df = pd.read_csv(f'../results/{name}.csv')
  row = df[df['type'] == 'test'].drop(columns=['epoch', 'type', 'loss']).iloc[0].to_dict()
  results[name] = row

df = pd.DataFrame(results).T
print(df)

                   accuracy  precision    recall        f1         wer
tiny               0.794844   0.951423  0.948248  0.949651    0.403611
tiny_ft            0.808234   0.956592  0.947597  0.951956    1.481541
tiny_ss            0.734191   0.923143  0.161783  0.150355   98.054060
tiny_ss_ft         0.755931   0.920729  0.178288  0.170569  105.189646
tiny_ss_ft_freeze  0.745897   0.930025  0.321666  0.416848   69.541284
tiny_ft_freeze     0.799433   0.953935  0.947164  0.950358    1.106143
